Phân loại văn bản với spaCy. Trình phân loại sẽ phát hiện thư spam, một chức năng phổ biến trong hầu hết các ứng dụng email.

In [13]:
import pandas as pd
import chardet

with open('spam.csv', 'rb') as f:
    result = chardet.detect(f.read())

# Đọc dataset
# spam: các đoạn văn bản spam, ham: văn bản bình thường (non-spam)
spam = pd.read_csv('spam.csv', encoding=result['encoding'])
spam.head(50) # 50 dòng đầu tiên

,label,text,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN
5,spam,FreeMsg Hey there darling it's been 3 week's n...,NaN,NaN,NaN
6,ham,Even my brother is not like to speak with me. ...,NaN,NaN,NaN
7,ham,As per your request 'Melle Melle (Oru Minnamin...,NaN,NaN,NaN
8,spam,WINNER!! As a valued network customer you have...,NaN,NaN,NaN
9,spam,Had your mobile 11 months or more? U R entitle...,NaN,NaN,NaN


In [14]:
import spacy

# Tạo một model bằng spacy
nlp = spacy.blank("en")

# Thêm TextCategorizer vào model
textcat = nlp.add_pipe("textcat")

In [15]:
# Thêm label vào phân loại text
textcat.add_label("ham")
textcat.add_label("spam")

1

Training a Text Categorizer Model

Tiếp theo, bạn sẽ chuyển đổi các nhãn trong dữ liệu thành dạng mà TextCategorizer yêu cầu. Đối với mỗi tài liệu, bạn sẽ tạo một từ điển của các giá trị boolean cho mỗi lớp.

Ví dụ, nếu một văn bản là “ham”, chúng ta cần một từ điển {‘ham’: True, ‘spam’: False}. Mô hình đang tìm kiếm những nhãn này bên trong một từ điển khác với khóa ‘cats’.

In [16]:
train_texts = spam['text'].values
train_labels = [{'cats': {'ham': label == 'ham',
                          'spam': label == 'spam'}} 
                for label in spam['label']]

Sau đó, kết hợp các text và label vào một list

In [17]:
train_data = list(zip(train_texts, train_labels))
train_data[:3]

[('Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...',
  {'cats': {'ham': True, 'spam': False}}),
 ('Ok lar... Joking wif u oni...', {'cats': {'ham': True, 'spam': False}}),
 ("Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's",
  {'cats': {'ham': False, 'spam': True}})]

Bây giờ bạn đã sẵn sàng để huấn luyện mô hình. Đầu tiên, tạo một bộ tối ưu hóa bằng cách sử dụng nlp.begin_training(). spaCy sử dụng bộ tối ưu hóa này để cập nhật mô hình. Nói chung, việc huấn luyện mô hình theo các lô nhỏ hơn thường hiệu quả hơn. spaCy cung cấp hàm minibatch trả về một generator tạo ra các lô nhỏ cho quá trình huấn luyện. Cuối cùng, các lô nhỏ được chia thành văn bản và nhãn, sau đó được sử dụng với nlp.update để cập nhật các tham số của mô hình.

In [18]:
from spacy.util import minibatch
from spacy.training.example import Example

# Đặt hạt giống ngẫu nhiên để đảm bảo tính nhất quán của kết quả huấn luyện.
spacy.util.fix_random_seed(1)

# Bắt đầu quá trình huấn luyện và tạo một bộ tối ưu hóa.
optimizer = nlp.begin_training()

# Tạo một generator cho các lô dữ liệu huấn luyện với kích thước lô là 8.
batches = minibatch(train_data, size=8)

Duyệt qua từng lô dữ liệu. Với mỗi lô, chúng ta duyệt qua từng cặp văn bản và nhãn, tạo một tài liệu từ văn bản, sau đó tạo một ví dụ từ tài liệu và nhãn. Cuối cùng, chúng ta cập nhật mô hình với ví dụ này sử dụng bộ tối ưu hóa đã tạo.

In [19]:
for batch in batches:
    for text, labels in batch:
        doc = nlp.make_doc(text)
        example = Example.from_dict(doc, labels)
        nlp.update([example], sgd=optimizer)

Đây chỉ là một vòng lặp huấn luyện (thời kỳ) qua dữ liệu. Mô hình thường cần nhiều thời kỳ. Sử dụng thêm vòng lặp cho nhiều thời kỳ hơn, và tùy chọn có thể xáo trộn lại dữ liệu huấn luyện ở đầu mỗi vòng lặp.

In [21]:
import random

# Đặt hạt giống ngẫu nhiên để đảm bảo tính nhất quán của kết quả huấn luyện.
random.seed(1)
spacy.util.fix_random_seed(1)

# Bắt đầu quá trình huấn luyện và tạo một bộ tối ưu hóa.
optimizer = nlp.begin_training()

# Khởi tạo một từ điển để theo dõi mất mát trong quá trình huấn luyện và 
# thiết lập một vòng lặp để huấn luyện mô hình trong 3 thời kỳ.

losses = {}
for epoch in range(3):
    random.shuffle(train_data)
    # Tạo một generator cho các lô dữ liệu huấn luyện với kích thước lô là 8.
    batches = minibatch(train_data, size=8)
    # Iterate through minibatches
    for batch in batches:
        for text, labels in batch:
            doc = nlp.make_doc(text)
            example = Example.from_dict(doc, labels)
            nlp.update([example], sgd=optimizer, losses=losses)
    print(losses)

{'textcat': 147.3330897448211}
{'textcat': 203.43055121840374}
{'textcat': 238.81971315683418}


Bây giờ bạn đã có một mô hình đã được huấn luyện, bạn có thể dự đoán với phương thức predict(). Văn bản đầu vào cần được tách thành từ vựng với nlp.tokenizer. Sau đó, bạn truyền các token vào phương thức predict để nhận lại điểm số. Điểm số này chính là xác suất mà văn bản đầu vào thuộc về các lớp.

In [24]:
texts = ["Had your mobile 11 months or more? U R entitled to Update to the latest colour mobiles with camera for Free! Call The Mobile Update Co FREE on 08002986030"]
docs = [nlp.tokenizer(text) for text in texts]
    
# Use textcat to get the scores for each doc
textcat = nlp.get_pipe('textcat')
scores = textcat.predict(docs)

print(scores)

[[1.3661123e-07 9.9999988e-01]]


Điểm số được sử dụng để dự đoán một lớp hoặc nhãn duy nhất bằng cách chọn nhãn có xác suất cao nhất. Bạn lấy chỉ số của xác suất cao nhất với scores.argmax, sau đó sử dụng chỉ số này để lấy chuỗi nhãn từ textcat.labels.

In [25]:
# Từ điểm số, tìm nhãn có điểm / xác suất cao nhất
predicted_labels = scores.argmax(axis=1)
print([textcat.labels[label] for label in predicted_labels])

['spam']
